In [ ]:
import pandas as pd
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
sns.set_style('white')

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
from bokeh.io import output_notebook

output_notebook()

### Get proportion of Nones in Variable col for each station
The Variable column contains the name of the sensor within station. 
Each station contains 15 sensors, split across 5 depths and 3 holes

In [ ]:
import pyarrow.parquet as pq
parquet_file = pq.ParquetFile('s3://uscrn-soilmoisture-ml/shiva/hydra/parquet/my_parquet.parquet')
tab = parquet_file.read(columns=['station_ID','Variable']).to_pandas()

In [ ]:
# Get all varaibles for a station
variable_df = tab.groupby(['station_ID'])['Variable'].unique()

In [ ]:
res = tab.groupby('station_ID').agg({'Variable': lambda x: x.isnull().mean()})
print(res)

In [ ]:
#Stations with no Variable data 
station_no_variable = list(res[res.Variable==1.0].index)
print(station_no_variable)

In [ ]:
# Station with all variable data
station_all_variable = list(res[res.Variable==0.0].index)
print(station_all_variable)

In [ ]:
station_major_variable = list(res[res.Variable<0.1].index)
print(len(station_major_variable))
print(station_major_variable)

In [ ]:
# All unique values in Variable column
tab['Variable'].unique()

In [ ]:
print(len(tab['station_ID'].unique()))
print(tab['station_ID'].unique())

In [ ]:
# STations

station_ids = """1001 1002 1003 1004 1005 1006 1007 1008 1009 1010 1011 1012 1013 1014
 1015 1017 1018 1019 1020 1021 1022 1023 1024 1025 1026 1027 1028 1029
 1030 1031 1032 1033 1034 1035 1036 1037 1038 1039 1040 1041 1042 1043
 1045 1046 1047 1049 1050 1066 1067 1068 1085 1105 1106 1107 1108 1109
 1110 1111 1112 1113 1114 1115 1116 1117 1118 1119 1120 1121 1122 1123
 1124 1125 1128 1129 1130 1138 1139 1140 1141 1142 1143 1144 1145 1166
 1186 1187 1231 1232 1233 1234 1235 1236 1266 1267 1268 1269 1296 1297
 1306 1307 1326 1327 1328 1329 1346 1347 1366 1386 1387 1406 1407 1426
 1446 1447 1467 1468 1487 1507 1508 1509 1510 1528 1529 1530 1531 1532
 1550 1551 1552 1570 1590 1610 1611 1612 1613 1630 1631 1632 1650 1651
 1652 1653 1654 1655 1656 1657 1658 1660 1661 1670 1671 1691 1692 1694
 1695 1696 1697 1698 1701 1702 1703 1704 1705 1706 1707 1708 1710 1711
 1712 1713 1714 1715 1716 1717 1721 1722 1726 1727 1728 1729 1730 1731
 1733 1734 1735 1736 1737 1738 1739 1740 1741 1742 1743 1744 1745 1746
 1747 1748 1749 1750 1753 1754 1755 1756 1771 1772 1773 1774 1775 1776
 1777 1778 1779 1780 1781 1782 1783 1784 1785 1786 1787 1788 1789 1790
 1791 1792 1793 1794 1795 1796 1797 1798 1799 1800 1801""".split(" ")

In [ ]:
station_ids = list(map(int, station_ids))

In [ ]:
#Station with soil moisture data
good_stations = list(set(station_ids) - set(station_no_variable))
print(len(good_stations))
print(good_stations)

### Get proportion of flagged data for each station

In [ ]:
good_stations = [1024, 1025, 1026, 1027, 1028, 1029, 1030, 1031, 1032, 1033, 1034, 1035, 1036, 1037, 1038, 1039, 1040, 1041, 1042, 1043, 1018, 1045, 1046, 1047, 1021, 1049, 1050, 1570, 1019, 1066, 1067, 1068, 1590, 1085, 1020, 1610, 1611, 1612, 1613, 1105, 1106, 1107, 1108, 1109, 1110, 1111, 1113, 1114, 1115, 1116, 1117, 1118, 1119, 1120, 1121, 1122, 1123, 1124, 1125, 1128, 1129, 1130, 1138, 1139, 1140, 1141, 1142, 1144, 1145, 1231, 1232, 1233, 1234, 1235, 1236, 1753, 1772, 1797, 1306, 1307, 1346, 1347, 1386, 1387, 1406, 1407, 1426, 1446, 1447, 1467, 1468, 1487, 1507, 1508, 1510, 1001, 1002, 1003, 1004, 1005, 1006, 1009, 1010, 1011, 1012, 1013, 1014, 1015, 1528, 1529, 1530, 1531, 1532, 1017, 1022, 1023]

In [ ]:
#Get the percentage of flags in each station

for i,id in enumerate(good_stations):
    filter_condition = ('station_ID', '==', id)
    df = pd.read_parquet('s3://uscrn-soilmoisture-ml/shiva/hydra/parquet/my_parquet.parquet',engine='pyarrow',filters=[filter_condition])
    flags = ['AutoFlag']
    count_dict =  {}
    count_dict['station_id'] = id
    for flag in flags:
        count_dict[flag] = (1-df[flag].value_counts(normalize=True).iloc[0])
    if i == 0:
        flags.append('station_id')
        count_df = pd.DataFrame([count_dict],columns = flags)
        flags.remove('station_id')
    else:
        temp_df = pd.DataFrame([count_dict])
        count_df = pd.concat([count_df, temp_df], ignore_index=True)
        del [temp_df]
        gc.collect()

In [ ]:
print(count_df)

In [ ]:
#For each station and variable
for i,id in enumerate(good_stations):
    filter_condition = ('station_ID', '==', id)
    df = pd.read_parquet('s3://uscrn-soilmoisture-ml/shiva/hydra/parquet/my_parquet.parquet',engine='pyarrow',filters=[filter_condition])
    grouped_df = df.groupby(['station_ID', 'Variable'])['AutoFlag'].sum().reset_index()
    zero_flags = grouped_df[grouped_df['AutoFlag'] == 0].reset_index(drop = True)
    if i == 0:
        count_df = zero_flags
    else:
        count_df = pd.concat([count_df, zero_flags], ignore_index=True)
    del [grouped_df]
    del [zero_flags]
    gc.collect()

In [ ]:
 station_variables_df = count_df.groupby('station_ID')['Variable'].apply(list).reset_index()

In [ ]:
#Stations and sensors with no flagged data
print(station_variables_df)

In [ ]:
#Checking that no other flag column is flagged
import pyarrow.parquet as pq
parquet_file = pq.ParquetFile('s3://uscrn-soilmoisture-ml/shiva/hydra/parquet/my_parquet.parquet')
flags = ['AutoFlag','NoPrcpResponse','FrozenRecovery','Noise','Failure','Static','Erratic','DiurnalNoise','Toohigh','Scaling','Zero','Spike']
for flag in flags:
    tab = parquet_file.read(columns=[flag]).to_pandas()
    print("Flag column: ", flag)
    print(tab[flag].unique())
    print('-'*20)

## Basic preprocessing

In [ ]:
def extract_depth(row):
    return int(row[-4:])

def any_flags(row):
    count =  row['AutoFlag'] + row['NoPrcpResponse'] + row['FrozenRecovery'] + row['Noise'] + row['Failure'] + row['Static'] + row['Erratic'] + row['DiurnalNoise'] + row['Toohigh'] + row['Scaling'] + row['Zero'] + row['Spike']
    if count == 0:
        return 0
    else:
        return 1



In [ ]:
import pyarrow as pa
import pyarrow.parquet as pq
source_parquet_file = 's3://uscrn-soilmoisture-ml/shiva/hydra/parquet/my_parquet.parquet'
target_parquet_file = 's3://uscrn-soilmoisture-ml/shiva/hydra/parquet/preprocessed_1.parquet'
columns = ['WBAN','station_ID','UTC_Start','Temperature','Precipitation','Tipping_Bucket','Wetness1','Wetness2','Variable','Depth','Dielectric','Volumetric','FlagBit','AutoFlag','NoPrcpResponse','FrozenRecovery','Noise','Failure','Static','Erratic','DiurnalNoise','Toohigh','Scaling','Zero','Spike']

for i,id in enumerate(good_stations):
    filter_condition = ('station_ID', '==', id)
    df = pd.read_parquet('s3://uscrn-soilmoisture-ml/shiva/hydra/parquet/my_parquet.parquet',engine='pyarrow',filters=[filter_condition])
    #df['Variable'].fillna(method='ffill', inplace=True)
    df["Variable"] = df["Variable"].ffill()
    df['Variable'] = df['Variable'].apply(extract_depth)
    df['Flag'] = df.apply(lambda x: any_flags(x), axis=1)
    df.drop(['NoPrcpResponse','FrozenRecovery','Noise','Failure','Static','Erratic','DiurnalNoise','Toohigh','Scaling','Zero','Spike'],axis=1,inplace=True)
    if i == 0:
        # Guess the schema of the CSV file from the first chunk
        parquet_schema = pa.Table.from_pandas(df=df).schema
        parquet_writer = pq.ParquetWriter(target_parquet_file, parquet_schema, compression='snappy')
    table = pa.Table.from_pandas(df, schema=parquet_schema)
    parquet_writer.write_table(table)
    del [df]
    

In [ ]:
parquet_writer.close()

In [ ]:
colums = ['station_id','count']
for id in station_ids:
    dict = {}
    dict['count'] =[]
    dict['station_id'] = []
    filter_condition = ('station_ID', '==', id)
    df = pd.read_parquet('s3://uscrn-soilmoisture-ml/shiva/hydra/parquet/my_parquet.parquet',engine='pyarrow',filters=[filter_condition])
    dict['station_id'].append(id)
    dict['count'].append(len(df))
    
    

### Time gap investigation
Followed by exploration of 3 different time series preprocessing techniques

In [ ]:
def check_time_gaps(row):
    filter_condition = ('station_ID', '==', row['station_ID'])
    station = pd.read_parquet('s3://uscrn-soilmoisture-ml/shiva/hydra/parquet/my_parquet.parquet',engine='pyarrow',columns=['station_ID','UTC_Start','Variable'],filters=[filter_condition])
    #station = df[df['station_ID'] == row['station_ID']].copy()
    no_gap, one_hr_gap, two_hr_gap, five_hr_gap = [], [], [], []

    for var in row['Variable'][:-1]:
        sensor = station[station['Variable'] == var]
        sensor_sorted = sensor.sort_values('UTC_Start')
        diff = sensor_sorted['UTC_Start'].diff()
        one_hr_gap_exists = any(diff > pd.to_timedelta('1 hour'))
        two_hr_gap_exists = any(diff > pd.to_timedelta('2 hours'))
        five_hr_gap_exists = any(diff > pd.to_timedelta('5 hours'))


        if not one_hr_gap_exists:
            no_gap.append(var)
        elif five_hr_gap_exists:
            five_hr_gap.append(var)
        elif two_hr_gap_exists:
            two_hr_gap.append(var)
        else:
            one_hr_gap.append(var)
    del [station]
    return pd.Series([no_gap, one_hr_gap, two_hr_gap, five_hr_gap])

result_df = station_variables_df.apply(lambda row: check_time_gaps(row), axis=1, result_type='expand')
result_df.columns = ['NoGap', '1HrGap', '2HrGap', '5HrGap+']

In [ ]:
time_gap_df = station_variables_df.join(result_df)
time_gap_df

### Differencing


In [ ]:
import seaborn as sns
sns.set_style('white')
from bokeh.palettes import HighContrast3
from bokeh.plotting import figure, show
from bokeh.layouts import row, column
from bokeh.models import ColumnDataSource, RangeTool
from bokeh.transform import factor_cmap, factor_mark

def plot_differenced_series(station, sensor,col,df,diff=0):
    #Replotting with Bokeh
    s1 = figure(title=f'Station {station} sensor {sensor} original data',width=1250, height=450, background_fill_color="#fafafa",x_axis_type="datetime")
    Flag = ['0','1']
    df.Flag = df.Flag.astype('str')
    s1.scatter("UTC_Start", col, source=df ,size=4, alpha=0.5, marker="circle",color=factor_cmap("Flag", 'Category10_3', Flag))
    
    df['diffrenced_series'] = df[col].diff(periods=diff)
    print(df.columns)
    s2 = figure(title=f'Station {station} sensor {sensor} Differenced {diff} order',width=1250, height=450, background_fill_color="#fafafa",x_axis_type="datetime")
    s2.scatter("UTC_Start","diffrenced_series",source=df, size=4, alpha=0.5,marker="circle",color=factor_cmap("Flag", 'Category10_3', Flag))
    print('after 2 obj')
    # put the results in a row and show
    show(column(s1, s2))
    

In [ ]:
target_parquet_file = 's3://uscrn-soilmoisture-ml/shiva/hydra/parquet/preprocessed_1.parquet'
stations = [1017, 1023,1528]

In [ ]:
for station in stations:
    filter_condition = ('station_ID', '==', station)
    df = pd.read_parquet(target_parquet_file, engine='pyarrow',filters=[filter_condition])
    print(station)
    variables = df['Variable'].unique()
    print(variables)
    for sensor in variables:
        plot_differenced_series(station,sensor,'Volumetric',df[df.Variable == sensor],1)
        break
    break
        

### Fourier Transform

In [ ]:
from bokeh.palettes import HighContrast3
from bokeh.plotting import figure, show
def fourier_transform(station, sensor):
    filter_condition = [('station_ID', '==', station),('Variable','==',sensor)]
    target_parquet_file = 's3://uscrn-soilmoisture-ml/shiva/hydra/parquet/preprocessed_1.parquet'
    sensor_df = pd.read_parquet(target_parquet_file, engine='pyarrow',filters=filter_condition)
    sensor_df.fillna(0,inplace=True)
    #print(sensor_df.head())
    ft = np.fft.fft(sensor_df['Volumetric'])
    frequencies = np.fft.fftfreq(len(sensor_df['Volumetric']), d=0.1)
    print('ft: ',len(ft))
    print('freq: ', len(frequencies))
    plot = figure(title=f"Frequency Domain for Station {station} and Sensor {sensor}")
    plot.line(frequencies, np.abs(ft))#, size=10, marker="circle")
    show(plot)
    #plt.plot(frequencies, np.abs(ft))
    #plt.title(f"Frequency Domain for Station {station} and Sensor {sensor}")
    #plt.ylim([-1, 10])
    #plt.xlim([0, 5000])
    #plt.xlabel("Frequency")
    #plt.ylabel("Amplitude")
    #plt.show()

    return ft, frequencies, sensor_df['UTC_Start'], sensor

In [ ]:
def remove_freq(threshold, ft, freqs, dates, sensor):
    threshold_frequency = threshold
    ft[np.abs(freqs) == threshold_frequency] = 0
    cleaned_signal = np.fft.ifft(ft)
    plt.figure(figsize=(10, 16))
    plot = figure(title=f"Sensor {sensor} (After Denoising), Threshold {threshold}",x_axis_type="datetime")
    plot.scatter(dates, np.real(cleaned_signal), size=2, marker="circle", alpha=0.4)
    show(plot)
    
    #sns.scatterplot(x=dates, y=np.real(cleaned_signal), s=5, alpha=0.5)
    #plt.title(f"Sensor {sensor} (After Denoising), Threshold {threshold}")
    #plt.xlabel("Time")
    #plt.ylabel("Volumetric Amplitude")
    #plt.show()

In [ ]:
#Station 1017, sensor 1005 
ft, frequencies, dates, sensor = fourier_transform(1017, 1005)
remove_freq(1, ft, frequencies, dates, sensor)

### Rolling averages/std

In [ ]:
def rolling_calculations(df):
    df = df.sort_values(by=['UTC_Start'])
    df['RollingAvg'] = df['Volumetric'].rolling(window_size, min_periods=window_size, center = True).mean()
    df['RollingStd'] = df['Volumetric'].rolling(window_size, min_periods=window_size, center = True).std()
    return df

In [ ]:
window_size=12
station = 1017

filter_condition = [('station_ID', '==', station)]
target_parquet_file = 's3://uscrn-soilmoisture-ml/shiva/hydra/parquet/preprocessed_1.parquet'
sensor_df = pd.read_parquet(target_parquet_file, engine='pyarrow',filters=filter_condition)
#windows = [13,25,49]
result_df = sensor_df.groupby(['station_ID', 'Variable']).apply(rolling_calculations)
result_df = result_df.droplevel([0,1])
sensor_df[f"StVol_{window_size}hrs"] = (result_df['Volumetric'] - result_df['RollingAvg']) / result_df['RollingStd']

plt.figure(figsize=(10, 6))
sns.scatterplot(x = 'UTC_Start', y = f"StVol_{window_size}hrs", data = sensor_df, s = 5, alpha=0.5)
plt.title(f'{station} {sensor} {window_size}', fontsize = 30)
plt.xlabel('Date', fontsize = 20)
plt.xticks(rotation = 30)
plt.ylabel(f"StVol_{window_size}hrs", fontsize = 20)


In [ ]:
from bokeh.plotting import figure, show
from bokeh.layouts import row, column
from bokeh.models import ColumnDataSource, RangeTool
from bokeh.transform import factor_cmap, factor_mark

def plot_rolling_avg(station, sensor, window_size):
    filter_condition = [('station_ID', '==', station),('Variable','==',sensor)]
    target_parquet_file = 's3://uscrn-soilmoisture-ml/shiva/hydra/parquet/preprocessed_1.parquet'
    sensor_df = pd.read_parquet(target_parquet_file, engine='pyarrow',filters=filter_condition)
    
    #windows = [13,25,49]
    result_df = sensor_df.groupby(['station_ID', 'Variable']).apply(rolling_calculations)
    result_df = result_df.droplevel([0,1])
    sensor_df[f"StVol_{window_size}hrs"] = (result_df['Volumetric'] - result_df['RollingAvg']) / result_df['RollingStd']
    Flag = ['0','1']
    sensor_df.Flag = sensor_df.Flag.astype('str')
    #Replotting with Bokeh
    s1 = figure(title=f'Station: {station} Sensor: {sensor} window_size: {window_size}',width=1250, height=450, background_fill_color="#fafafa",x_axis_type="datetime",x_range=(sensor_df.UTC_Start[1500], sensor_df.UTC_Start[2500]))
    s1.scatter("UTC_Start", f"StVol_{window_size}hrs", source=sensor_df,size=4, alpha=0.5, marker="circle",color=factor_cmap("Flag", 'Category10_3', Flag))

    select = figure(title="Drag the middle and edges of the selection box to change the range above",
                height=130, width=800, y_range=s1.y_range,
                x_axis_type="datetime", y_axis_type=None,
                tools="", toolbar_location=None, background_fill_color="#efefef")

    range_tool = RangeTool(x_range=s1.x_range)
    range_tool.overlay.fill_color = "navy"
    range_tool.overlay.fill_alpha = 0.2
    
    select.scatter("UTC_Start", f"StVol_{window_size}hrs",source=sensor_df,color=factor_cmap("Flag", 'Category10_3', Flag))
    select.ygrid.grid_line_color = None
    select.add_tools(range_tool)

    # put the results in a row and show
    show(column(s1,select))
    '''
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x = 'UTC_Start', y = f"StVol_{window_size}hrs", data = sensor_df, s = 5, alpha=0.5)
    plt.title(f'{station} {sensor} {window_size}', fontsize = 30)
    plt.xlabel('Date', fontsize = 20)
    plt.xticks(rotation = 30)
    plt.ylabel(f"StVol_{window_size}hrs", fontsize = 20)
    '''
window_size=72
plot_rolling_avg(1017,1005,window_size)

In [ ]:
window_size=168
plot_rolling_avg(1017,1005,window_size)

### Climatology preprocessing

In [ ]:
#For a given month-day-hour, take all points across all years and +- 2 days (window), calculate the mean/std and use this to standardize that point
target_parquet_file = 's3://uscrn-soilmoisture-ml/shiva/hydra/parquet/preprocessed_1.parquet'
stations = [1017, 1023,1528]

In [ ]:
from datetime import datetime, timedelta
filter_condition = ('station_ID', '==', 1017)
df = pd.read_parquet(target_parquet_file, engine='pyarrow',filters=[filter_condition])

In [ ]:
from datetime import datetime, timedelta

def preprocessing(station,sensor, column, df, window):
    df_1 = df[df.Variable==sensor]
    df_1.loc[:,['month']] = df_1['UTC_Start'].dt.month
    df_1.loc[:,['day']] = df_1['UTC_Start'].dt.day
    df_1.loc[:,['hour']] = df_1['UTC_Start'].dt.hour
    climatology_dict = {}
    window = window
    td = timedelta(days=window)
    start_date = datetime.strptime('01-01-00','%m-%d-%H')
    end_date = datetime.strptime('12-31-23','%m-%d-%H')
    print(start_date.strftime('%m-%d-%H'))
    it = timedelta(hours=1)
    while start_date <= end_date:
        window_date_start = start_date - td
        temp_df = pd.Series()
        while window_date_start <= start_date + td:
            #print(df[(df.month == window_date_start.month) & (df.day == window_date_start.day) & (df.hour == window_date_start.hour)].Volumetric)
            temp_df = pd.concat([df_1[(df_1.month == window_date_start.month) & (df_1.day == window_date_start.day) & (df_1.hour == window_date_start.hour)][column], temp_df], ignore_index=True)
            window_date_start = window_date_start + td
        climatology_dict[start_date.strftime('%m-%d-%H')] = (temp_df.mean(), temp_df.std(), temp_df.isna().mean())
        start_date = start_date + it
        del [temp_df]

    #For Leap year
    start_date = datetime.strptime('2012-02-29-00','%Y-%m-%d-%H')
    end_date = datetime.strptime('2012-02-29-23','%Y-%m-%d-%H')
    it = timedelta(hours=1)
    while start_date <= end_date:
        window_date_start = start_date - td
        temp_df = pd.Series()
        while window_date_start <= start_date + td:
            #print(df[(df.month == window_date_start.month) & (df.day == window_date_start.day) & (df.hour == window_date_start.hour)].Volumetric)
            temp_df = pd.concat([df_1[(df_1.month == window_date_start.month) & (df_1.day == window_date_start.day) & (df_1.hour == window_date_start.hour)]['Volumetric'], temp_df], ignore_index=True)
            window_date_start = window_date_start + td
        climatology_dict[start_date.strftime('%m-%d-%H')] = (temp_df.mean(), temp_df.std(), temp_df.isna().mean())
        start_date = start_date + it
        del [temp_df]
    return climatology_dict


In [ ]:
from bokeh.palettes import HighContrast3
from bokeh.plotting import figure, show
from bokeh.layouts import row, column
from bokeh.models import ColumnDataSource, RangeTool
from bokeh.transform import factor_cmap, factor_mark

def plot_preprocessed(df, station, sensor):

    #palette = d3['Category10'][len(df['Flag'].unique())]
    #color_map = bmo.CategoricalColorMapper(factors=df['Flag'].unique(),palette=palette)
    #Replotting with Bokeh
    #source = ColumnDataSource.from_df(df)
    Flag = ['0','1']
    df.Flag = df.Flag.astype('str')
    s1 = figure(title=f'Station {station} sensor {sensor}',width=1250, height=450, background_fill_color="#fafafa",x_axis_type="datetime")
    s1.scatter('UTC_Start', 'Volumetric',  source = df, size=4, alpha=0.5, marker="circle", legend_field='Flag',color=factor_cmap("Flag", 'Category10_3', Flag))
    
    s2 = figure(title=f'Station {station} sensor {sensor} preprocessed',width=1250, height=450, background_fill_color="#fafafa",x_axis_type="datetime")
    s2.scatter('UTC_Start','preprocessed_volumetric',source = df, size=4, alpha=0.5,marker="circle", legend_field='Flag',color=factor_cmap("Flag", 'Category10_3', Flag))
    print('after 2 obj')
    # put the results in a row and show
    show(column(s1, s2))
    

In [ ]:
def func(row):
    return (row['Volumetric'] - climatology_dict[row['preprocess_key']][0])/climatology_dict[row['preprocess_key']][1]

for station in stations:
    filter_condition = ('station_ID', '==', station)
    df = pd.read_parquet(target_parquet_file, engine='pyarrow',filters=[filter_condition])
    variables = df['Variable'].unique()
    print(variables)
    for sensor in variables:
        df_1 = df[df.Variable==sensor]
        climatology_dict = preprocessing(station,sensor,'Volumetric',df, 1)
        df_1.loc[:,['preprocess_key']] = df_1.UTC_Start.dt.strftime('%m-%d-%H')
        df_1.loc[:,['preprocessed_volumetric']] = df_1.apply(lambda row: func(row), axis=1)
        plot_preprocessed(df_1, station, sensor)
        break
    break

In [ ]:
climatology_dict